## Collecting IOCs to Detect Encrypted DNS

There has been some [attention](https://www.securitymagazine.com/articles/91674-disappearing-dns-dot-and-doh-where-one-letter-makes-a-great-difference) given lately to different protocols used to encrypt DNS traffic and their pros and cons with respect to privacy and security.  I have written several recent blog posts on the reality of the situation and what we, as a community, can do to reduce the risk associated with the use of any encrypted DNS protocol being used on our networks.

In one of those posts I talk about using the publicly known list of DNS servers that support DoT, DoH and DNSCrypt as IOCs to identify potential usage of encrypted DNS.  This list can be found [here](https://dnscrypt.info/public-servers/).  Once on that site you can click on each row in the list to view the server information including the IP and domain.

![dns stamps](dnsstampstable.png)

That's great and all, but I don't think anyone here wants to click through a bunch of tables and copy / paste IPs and Domains in order to get all of the destinations so we can add them to our IOC lists.  I figured that information in those tables must be coming from _somewhere_ so I started to poke around in the [dnscrypt github](https://github.com/dnscrypt) to see if I could find them.

It turns out that there is a [repo](https://github.com/DNSCrypt/dnscrypt-resolvers) that contains exactly what I was looking for.  According to the readme, the list of servers is located [in the repo](https://github.com/DNSCrypt/dnscrypt-resolvers/tree/master/v2), and also hosted [here](https://download.dnscrypt.info/dnscrypt-resolvers/v2/).

This immediately revealed a new issue, the format of the files is markdown, and the dns server information is in a format called [DNSStamps](https://dnscrypt.info/stamps-specifications/).  DNSStamps are the format used by various DNS Encryption tools (like dnscrypt-proxy), but they're not a convienent format for collecting IOCs from them.

![dns stamp example](dnsstampexample.png)

TLDR;

We found a source of structured text containing Domains and IPs of public DNS servers supporting encryption. However, that format is not convenient, lets convert it to something we can use for analysis.

### Shut Up and Show Me Code

Thankfully, there's already a [python module for parsing dns stamps](https://pypi.org/project/dnsstamps/), and it's very simple to use.  But first we'll need to download the files with the dnsstamps and get rid of lines we don't need.  We'll start with the first file: [onion-services.md](https://download.dnscrypt.info/dnscrypt-resolvers/v2/onion-services.md).  Most of my programming lately has been with functional languages, which is why you may notice the use of `filter` and `map` in favor of list comprehensions.

In [50]:
import requests

r = requests.get("https://download.dnscrypt.info/dnscrypt-resolvers/v2/onion-services.md")
t = r.text
t

"# DNS servers as .onion services\n\nAll DNSCrypt and DoH servers are accessible over Tor, via exit nodes.\nThis is safe as all the transactions are encrypted and authenticated.\n\nHowever, it may be faster to directly access a server as an onion\nservice. This requires specifically configured servers.\n\nThe servers below are not accessible without Tor.\n\nTo use that list, add this to the `[sources]` section of your\n`dnscrypt-proxy.toml` configuration file:\n\n    [sources.'onion-services']\n    urls = ['https://raw.githubusercontent.com/DNSCrypt/dnscrypt-resolvers/master/v2/onion-services.md', 'https://download.dnscrypt.info/resolvers-list/v2/onion-services.md']\n    minisign_key = 'RWQf6LRCGA9i53mlYecO4IzT51TGPpvWucNSCh1CBM0QTaLn73Y7GFO3'\n    cache_file = 'onion-services.md'\n\n--\n\n\n## onion-cloudflare\n\nCloudflare Onion Service\n\nsdns://AgcAAAAAAAAAACC0WWFtenR5met-s8i0oiShMtYstulWSybPBq-zBUEMNT5kbnM0dG9ycG5sZnMyaWZ1ejJzMnlmM2ZjN3JkbXNiaG02cnc3NWV1ajM1cGFjNmFwMjV6Z3FhZC5vbml

### Extracting Only the DNSStamps
Next we'll need to extract only the lines that have the dns stamps; those that start with `sdns://`.  Of course there's plenty of ways to go about this, but I found the following to be simple and effective enough (and I can't think of any obvious gotchas).

In [51]:
stamps = list(filter(lambda x: x.startswith('sdns://'), t.splitlines()))
stamps

['sdns://AgcAAAAAAAAAACC0WWFtenR5met-s8i0oiShMtYstulWSybPBq-zBUEMNT5kbnM0dG9ycG5sZnMyaWZ1ejJzMnlmM2ZjN3JkbXNiaG02cnc3NWV1ajM1cGFjNmFwMjV6Z3FhZC5vbmlvbgovZG5zLXF1ZXJ5']

### Parsing the Stamps
Now we'll used the dnsstamps python module to parse all the stamps in the output (which for this example is only one stamp).  If you don't already have dnsstamps installed you can run the following cell.

In [52]:
import sys
!{sys.executable} -m pip install dnsstamps

You should consider upgrading via the '/usr/local/Cellar/jupyterlab/2.1.0_1/libexec/bin/python3.8 -m pip install --upgrade pip' command.


In [25]:
import dnsstamps
ps = map(lambda x: dnsstamps.format(dnsstamps.parse(x)), stamps)
_ = list(ps)

DoH DNS stamp

DNSSEC: yes
No logs: yes
No filter: yes
IP Address: 
Hashes: [b'b459616d7a747999eb7eb3c8b4a224a132d62cb6e9564b26cf06afb305410c35']
Hostname: dns4torpnlfs2ifuz2s2yf3fc7rdmsbhm6rw75euj35pac6ap25zgqad.onion
Path: /dns-query
Bootstrap IPs: []

sdns://AgcAAAAAAAAAACC0WWFtenR5met-s8i0oiShMtYstulWSybPBq-zBUEMNT5kbnM0dG9ycG5sZnMyaWZ1ejJzMnlmM2ZjN3JkbXNiaG02cnc3NWV1ajM1cGFjNmFwMjV6Z3FhZC5vbmlvbgovZG5zLXF1ZXJ5


That will print out the decoded stamp values, which includes some of the information that we're after (although this one understandably doesn't include an IP since it's a `.onion` address).  Instead of printing it out, we can just use the object that `dnsstamps.parse()` returns, which can be seen in the source code [here](https://github.com/chrisss404/python-dnsstamps/blob/master/dnsstamps/formatter/formatter.py).

We'll try this out on the example that we currently have.

In [53]:
stamp = dnsstamps.parse(list(stamps)[0])
stamp

We can even use tab completion in the notebook to show us what the attributes are.  Some notebook-ception for you:
![tab completion](tabcompletion.png)

In [55]:
stamp.address

''

(note that if a field is empty it == '', and since '' is _falsey_ in python, we can use `None` instead of `!= ''`)

In [56]:
stamp.hostname

'dns4torpnlfs2ifuz2s2yf3fc7rdmsbhm6rw75euj35pac6ap25zgqad.onion'

Seems easy enough.  Let's write a function that will allow us do this for any of the files.  

In [57]:
def parseDnsStampFromUrl(url):
    r = requests.get(url)
    stamps = filter(lambda x: x.startswith('sdns://'),r.text.splitlines())
    parsed_stamps = list(map(dnsstamps.parse, stamps))
    domains = filter(None, map(lambda x: x.hostname, parsed_stamps))
    ips = filter(None, map(lambda x: x.address, parsed_stamps))
    return {'domains': list(domains),'ips': list(ips) }

Alright, let's try it on our first url:

In [58]:
parseDnsStampFromUrl("https://download.dnscrypt.info/dnscrypt-resolvers/v2/onion-services.md")

{'domains': ['dns4torpnlfs2ifuz2s2yf3fc7rdmsbhm6rw75euj35pac6ap25zgqad.onion'],
 'ips': []}

Awesome.  So now all we have to do is map that function over the list of urls from [the list of dnsstamp files](https://download.dnscrypt.info/dnscrypt-resolvers/v2/).  Of course, there's so few files that we _could_ just copy / paste them into this notebook... but what if they add new files?  And perhaps more importantly, where's the fun in that? Let's burden our code with another dependancy :D

In [35]:
!{sys.executable} -m pip install beautifulsoup4

     |████████████████████████████████| 109 kB 1.3 MB/s eta 0:00:01


In [59]:
from bs4 import BeautifulSoup
url = "https://download.dnscrypt.info/dnscrypt-resolvers/v2/"
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
hrefs = map(lambda x: x.get('href'),soup.find_all('a'))
md_files = filter(lambda x: x.endswith('.md'), hrefs)
list(md_files)

['onion-services.md',
 'opennic.md',
 'parental-control.md',
 'public-resolvers.md',
 'relays.md']

Alright, seems to work well.  Now lets functionize it.  In addition to what we did above we also need to append the original url to get the full path to each of the files.

In [60]:
def getDnsStampFileUrls(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    hrefs = map(lambda x: x.get('href'),soup.find_all('a'))
    md_files = filter(lambda x: x.endswith('.md'), hrefs)
    return map(lambda x: url + x, md_files)

Now all we have to do is add everything together.

In [61]:
url = "https://download.dnscrypt.info/dnscrypt-resolvers/v2/"
urls = getDnsStampFileUrls(url)
stamps = list(map(parseDnsStampFromUrl,urls))

And then pull out the domains and ips:

In [62]:
{'domains': sum((x['domains'] for x in stamps), []), 'ips': sum((x['ips'] for x in stamps), [])}

{'domains': ['dns4torpnlfs2ifuz2s2yf3fc7rdmsbhm6rw75euj35pac6ap25zgqad.onion',
  'ibksturm.synology.me',
  'doh.seby.io:8443',
  'doh-2.seby.io:443',
  'doh.familyshield.opendns.com',
  'family.cloudflare-dns.com',
  'doh.cleanbrowsing.org',
  'doh.cleanbrowsing.org',
  'dns.aa.net.uk',
  'dns.aaflalo.me',
  'dns-nyc.aaflalo.me',
  'dns-nyc.aaflalo.me',
  'dns.adguard.com',
  'dns-family.adguard.com',
  'dns.alidns.com',
  'dns.alekberg.net:443',
  'dohtrial.att.net',
  'dns.brahma.world',
  'dns.brahma.world',
  'doh.opendns.com',
  'doh.opendns.com',
  'dns.cloudflare.com',
  'family.cloudflare-dns.com',
  'dns.cloudflare.com',
  'security.cloudflare-dns.com',
  'security.cloudflare-dns.com',
  'commons.host',
  'dns.containerpi.com',
  'dns.containerpi.com',
  'odvr.nic.cz',
  'dns.digitale-gesellschaft.ch',
  'dns.digitale-gesellschaft.ch',
  'dns.digitale-gesellschaft.ch',
  'dns.digitale-gesellschaft.ch',
  'doh.dns.sb',
  'dns1.dnscrypt.ca:453',
  'dns1.dnscrypt.ca:453',
  'dns2

It looks like there's a tad more formatting we need to deal with.  Some of the destinations have ports tacked onto the end of them (`doh.seby.io:8443`) which may be useful if your security stack allows you to specify domains _and_ ports, but for now let's assume that not everyone has that ability.  It also appears that the ipv6 ips are surrounded by square brackets `[]`.  Rather than fill up the last line above with even more code, let's make our code a tad bit more manageable and just create some helper functions to help us filter.

In [63]:
def massageString(s):
    return s.replace('[','').split(']')[0] if s.startswith('[') else s.split(':')[0]

def sumStamps(stamps):
    domains = sum((list(map(massageString,x['domains'])) for x in stamps),[])
    ips = sum((list(map(massageString,x['ips'])) for x in stamps),[])
    return {'domains': set(domains), 'ips': set(ips)}


    

In [64]:
results = sumStamps(stamps)
results

{'domains': {'commons.host',
  'dns-family.adguard.com',
  'dns-nyc.aaflalo.me',
  'dns.aa.net.uk',
  'dns.aaflalo.me',
  'dns.adguard.com',
  'dns.alekberg.net',
  'dns.alidns.com',
  'dns.brahma.world',
  'dns.cloudflare.com',
  'dns.containerpi.com',
  'dns.digitale-gesellschaft.ch',
  'dns.google',
  'dns.nextdns.io',
  'dns.t53.de',
  'dns.twnic.tw',
  'dns1.dnscrypt.ca',
  'dns2.alekberg.net',
  'dns2.dnscrypt.ca',
  'dns4torpnlfs2ifuz2s2yf3fc7rdmsbhm6rw75euj35pac6ap25zgqad.onion',
  'dnsforge.de',
  'dnsse.alekberg.net',
  'doh-2.seby.io',
  'doh-de.blahdns.com',
  'doh-fi.blahdns.com',
  'doh-ipv6.crypto.sx',
  'doh-jp.blahdns.com',
  'doh-sg.blahdns.com',
  'doh.233py.com',
  'doh.appliedprivacy.net',
  'doh.centraleu.pi-dns.com',
  'doh.cleanbrowsing.org',
  'doh.crypto.sx',
  'doh.dns.sb',
  'doh.dnslify.com',
  'doh.eastus.pi-dns.com',
  'doh.familyshield.opendns.com',
  'doh.ffmuc.net',
  'doh.li',
  'doh.libredns.gr',
  'doh.northeu.pi-dns.com',
  'doh.opendns.com',
  'do

Now all that's left is to do is write those IOCs to a file.

In [65]:
with open('ips.txt','w') as f:
    f.writelines('\n'.join(results['ips']))
with open('domains.txt','w') as f:
    f.writelines('\n'.join(results['domains']))

Let's make sure it worked

In [66]:
!tail ips.txt

1.0.0.3
2a01:4f8:1c0c:8233::1
37.120.147.2
51.15.124.208
2606:4700:30::681c:6a
104.28.28.34
146.112.41.2
2a0d:2a00:1::
174.138.21.128
198.100.148.224

In [67]:
!tail domains.txt

dns.alidns.com
ibksturm.synology.me
dns.t53.de
dns.digitale-gesellschaft.ch
family.cloudflare-dns.com
doh.crypto.sx
dns.alekberg.net
doh.233py.com
dns.aaflalo.me
doh-sg.blahdns.com

Looks like we got it.  Now you can take that and feed it into whatever tools you have in order to block, or at least detect, usage of encrypted DNS on your networks.

If you have any questions, feel free to reach out to my barely followed twitter account [@SonicTheHexHog](https://twitter.com/SonicTheHexHog).  I'll post all the code below this to reduce the amount of neccesary copy/pasting.

In [78]:
import requests
import dnsstamps
from bs4 import BeautifulSoup

def parseDnsStampFromUrl(url):
    r = requests.get(url)
    stamps = filter(lambda x: x.startswith('sdns://'),r.text.splitlines())
    parsed_stamps = list(map(dnsstamps.parse, stamps))
    domains = filter(None, map(lambda x: x.hostname, parsed_stamps))
    ips = filter(None, map(lambda x: x.address, parsed_stamps))
    return {'domains': list(domains),'ips': list(ips) }

def getDnsStampFileUrls(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    hrefs = map(lambda x: x.get('href'),soup.find_all('a'))
    md_files = filter(lambda x: x.endswith('.md'), hrefs)
    return map(lambda x: url + x, md_files)

def massageString(s):
    return s.replace('[','').split(']')[0] if s.startswith('[') else s.split(':')[0]

def sumStamps(stamps):
    domains = sum((list(map(massageString,x['domains'])) for x in stamps),[])
    ips = sum((list(map(massageString,x['ips'])) for x in stamps),[])
    return {'domains': set(domains), 'ips': set(ips)}

def main():
    url = "https://download.dnscrypt.info/dnscrypt-resolvers/v2/"
    urls = getDnsStampFileUrls(url)
    stamps = list(map(parseDnsStampFromUrl,urls))
    results = sumStamps(stamps)
    with open('ips.txt','w') as f:
        f.writelines('\n'.join(results['ips']))
    with open('domains.txt','w') as f:
        f.writelines('\n'.join(results['domains']))

if __name__ == '__main__':
    main()
